HAI Card Game

In [2]:
import random
import numpy as np
from linear_irl import large_irl

In [3]:
def get_states(cards, height, memo = {}):
    if cards == [0] * height:
        return {f'0 ' * height}
    returnedCards = set()
    if tuple(cards) in memo:
        return memo[tuple(cards)]    
    for i, card in enumerate(cards):
        if card != 0:
            nextCards = cards.copy()
            nextCards[i] //= 10
            returnedCards.add(' '.join([str(i) for i in nextCards]))
            returnedCards |= get_states(nextCards, height, memo)
    memo[tuple(cards)] = set(returnedCards)
    return set(returnedCards)

In [4]:
states = get_states([131242, 412323, 142243, 443131], 4)

In [5]:
list(states).index('131242 412323 14224')

ValueError: '131242 412323 14224' is not in list